In [1]:
map_name = 'k_near_map'
# --- Generate Map --- #
import folium

center = [33.748997, -84.387985]
m = folium.Map(center, zoom_start=10)
m.save(f'./outputs/maps/{map_name}.html')

m = folium.Map(
    location=[33.748997, -84.387985],
    tiles='Stamen Toner',
    zoom_start=10
)

In [2]:
import pandas as pd
import geopandas
from shapely.geometry import shape,mapping, Point, Polygon, MultiPolygon
import matplotlib.pyplot as plt
# import geoplot
from project_tools import load_obj, interact, save_obj

def CreatePointMap(df, Point_name):
    # cleaning
    df[['Latitude','Longitude']] = df[['Latitude','Longitude']].apply(pd.to_numeric, errors='coerce')
    # --- Converts df type to GeoDataFrame --- #
    pointDF = df
    df['coordinates'] = df[['Latitude', 'Longitude']].values.tolist()
    df['coordinates'] = df['coordinates'].apply(Point)
    df = geopandas.GeoDataFrame(df, geometry='coordinates')
    df = df.dropna(subset=[Point_name,'Latitude', 'Longitude','coordinates'])
    return df

def GeneratePointMap(Names , Latitudes, Longitudes, color, radius):
    for i, Name in enumerate(Names.iterrows()):
        folium.CircleMarker(
            location=[Latitudes.iloc[i,0],Longitudes.iloc[i,0]],
            tooltip = str(Names.iloc[i,0]),
            radius= radius,
            popup=str(Names.iloc[i,0]),
            color=color,
            fill=True,
            fill_color=color 
        ).add_to(m)
        
def covertname(x):
    try:
        x=inv_matcher[x]
    except KeyError:
        try:
            x=map_[x]
        except KeyError:
            x=0
    return int(x)

def plotfromdf(df, name, color='blue', size=3):
    df = CreatePointMap(df, name)
    GeneratePointMap(df.loc[:,[name]], df.loc[:,['Latitude']],df.loc[:,['Longitude']], color, radius=size)

In [3]:
# map_ is a dict converting names to station numbers
map_=server_info = load_obj('map')
del map_['would_not_load']
del map_['loaded_not_a_name']
map_df = pd.DataFrame(map_, index=['vds_station_number']).T
# info is a dict with extended station info eg lat long
# mostly all formatting
info=server_info = load_obj('info')
info_df = pd.DataFrame(info).T
names = pd.read_csv('support//inventory_xpath.csv',usecols = [0]).names # col names
info_df.columns = names
info_df['names'] = info_df.index
info_df.index = pd.RangeIndex(len(info_df.index))

vds_info = pd.read_csv('data/vds/location_data/vds_info_total.csv')
matcher = dict(zip(list(vds_info['value']),list(vds_info['description'])))


# remove if not GDOT
delete = []
onlyGDOT = True
for key, val in matcher.items():
    if onlyGDOT:
        if val[0] != 'G':
            delete.append(key)
        else:
            matcher[key] = val.split(':')[0]
    else:
        matcher[key] = val.split(':')[0]
for key in delete:
    del matcher[key]
inv_matcher = {v:k for k, v in matcher.items()}
#convert ID name to number format
info_df['ID2'] = info_df['ID'].map(covertname)
# only the ones being scrapped
info_df = info_df[info_df['ID2']!=0]

# --- plot all GDOT VDS stations --- #
plotfromdf(info_df, 'ID2', color='blue', size=3)

In [4]:
CCS_info = pd.read_csv("./data/vds/location_data/ccs_info.csv")
CCS_info['fixed_idx'] = pd.RangeIndex(len(CCS_info.index))

# --- plot all CCS stations --- #
plotfromdf(CCS_info, 'fixed_idx', color='red', size=4)

In [ ]:
print(len(CCS_info.name.tolist()))
for name in CCS_info.name.tolist():
    name = name[5:]
    print(f'{name[:3]}-{name[3:]}')

In [ ]:
# keep_vds = list(set([6968,6957,6954,3477,4465,3478,4464,15801,15802,15799,4255,3576,3577,4255,4275,3556,15886,3555,
#        3553,3233,3234,3266,2793,3232,3266,3007,3227,3226,3271,3209,3421,4491,3420,4132,3419,3923,3417,
#        3441,3838,6176,4473,4521,4131,4130,3918,4518,4517,3174,3173,3916,4128,4127,3915,2941,3682,3681,
#        2942,4042,3398,4041,4045,3263,202,2980,2979,3652,3651,2999,2998,3634,3633,6296,4164,2914,2915,5597,
#        3890,4103,4102,3889,6957,6954,6968,46723,46707,16044,16020,16019,16043,46825,4643,4642,3808,3807,3778,
#        4674,4669,3782,2821,2822,3503,3504,3477,4465,3478,4464,15805,2809,2810,3609,2821,2822,3503,3504,
#        15891,15892,3444,3621,2798,2797,3622,2788,6173,6170,4136,3926,6954,4643,4642,3808,4177,4328,4329,4178,
#        4354,4210,5734,4211,13414,13415,13419,13416,13417,4317,3513,4316,13482,13508,13516,3522,3544,4287,4286,
#        3545,3551,3550,2907,4217,2906,6505,4815,4811,2901,2900,4223,4224,2896,2895,4357,2894,4359,2893,7138,
#        7150,7129,7185,7172,7070,7090,7079,3998,3386,3309,5882,5854,3653,3654,4028,3280,4024,4023,3285,3461,
#        4729,3462,4728,4510,3340,2948,3920,4815,4811,4847,4839,3778,4669,3778,4669,3776,4530,3498,2826,2827,
#        4407,2837,4406,2836,3954,3953,4058,4135,4134,3208,6184,6192,3157,6154,4905,4903,4643,4815,4811,3542,
#        4388,4389,2855,2856,5846,5942,5846,5918,5929,5917,5942,5908,5845,4394,4393,2852,5906,5905,5839,3778,
#        4674,3776,4530,4412,6068,6067,6070,6071,3491,2834,3490,2835,7150,7129,7138,7172,7070,4643,7090,6968,
#        6957,6954,
#        16073,16016,16040,16015,
#                 ]))


# print(len(keep_vds))
# selected = VDS_df[VDS_df['ID2'].isin(keep_vds)]
# print(len(selected))
# # --- POINT PLOT VDS --- #
# selected = CreatePointMap(selected, 'ID2')
# # draw points on map
# GeneratePointMap(selected.loc[:,['ID2']], selected.loc[:,['Latitude']],selected.loc[:,['Longitude']], 'green', radius=5)

In [9]:
keep_vds = list(set([6968,6957,6954,3477,4465,3478,4464,15801,15802,15799,4255,3576,3577,4255,4275,3556,15886,3555,
       3553,3233,3234,3266,2793,3232,3266,3007,3227,3226,3271,3209,3421,4491,3420,4132,3419,3923,3417,
       3441,3838,6176,4473,4521,4131,4130,3918,4518,4517,3174,3173,3916,4128,4127,3915,2941,3682,3681,
       2942,4042,3398,4041,4045,3263,202,2980,2979,3652,3651,2999,2998,3634,3633,6296,4164,2914,2915,5597,
       3890,4103,4102,3889,6957,6954,6968,46723,46707,16044,16020,16019,16043,46825,4643,4642,3808,3807,3778,
       4674,4669,3782,2821,2822,3503,3504,3477,4465,3478,4464,15805,2809,2810,3609,2821,2822,3503,3504,
       15891,15892,3444,3621,2798,2797,3622,2788,6173,6170,4136,3926,6954,4643,4642,3808,4177,4328,4329,4178,
       4354,4210,5734,4211,13414,13415,13419,13416,13417,4317,3513,4316,13482,13508,13516,3522,3544,4287,4286,
       3545,3551,3550,2907,4217,2906,6505,4815,4811,2901,2900,4223,4224,2896,2895,4357,2894,4359,2893,7138,
       7150,7129,7185,7172,7070,7090,7079,3998,3386,3309,5882,5854,3653,3654,4028,3280,4024,4023,3285,3461,
       4729,3462,4728,4510,3340,2948,3920,4815,4811,4847,4839,3778,4669,3778,4669,3776,4530,3498,2826,2827,
       4407,2837,4406,2836,3954,3953,4058,4135,4134,3208,6184,6192,3157,6154,4905,4903,4643,4815,4811,3542,
       4388,4389,2855,2856,5846,5942,5846,5918,5929,5917,5942,5908,5845,4394,4393,2852,5906,5905,5839,3778,
       4674,3776,4530,4412,6068,6067,6070,6071,3491,2834,3490,2835,7150,7129,7138,7172,7070,4643,7090,6968,
       6957,6954,
       16073,16016,16040,16015,
                ]))


print(len(keep_vds))
selected = info_df[info_df['ID2'].isin(keep_vds)]
print(len(selected))

selected['dis'] = selected['ID2'].astype(str)+' | '+selected['Direction']+' | '+selected['Roadway_Name']+' | '+selected['Roadway_Name']

# --- plot all CCS stations --- #
plotfromdf(selected, 'dis', color='green', size=6)

237
236


/home/clint/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/clint/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/home/clint/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [ ]:
selected

In [ ]:
CCS_df['fixed_idx'] = CCS_df['fixed_idx'].map(lambda x: int(x))

keep_ccs = list(set([92,91,101,100,102,105,104,103,194,302,283,113,120,96,182,203,196,198,201,181,
                     184,185,180,197,200,176,186,179,86,88,87,191,136,137,115,246,216,219,218,214,217,
                     224,225,222,141,135,132,133,139,122,190,122,190,177,178,178,165,193,293,294,324,
                     277,25,274,275,85,237,166,187,124,127,142,138,128,117,228,183,199,202,188,210,
                     234,160,235,236,81,125,140,134,126,189,
                ]))


print(len(keep_ccs))
selected = CCS_df[CCS_df['fixed_idx'].isin(keep_ccs)]
print(len(selected))
# --- POINT PLOT VDS --- #
selected = CreatePointMap(selected, 'fixed_idx')
# draw points on map
GeneratePointMap(selected.loc[:,['fixed_idx']], selected.loc[:,['Latitude']],selected.loc[:,['Longitude']], 'purple', radius=8)

In [10]:
m.save(f'./outputs/maps/{map_name}.html')

In [ ]:
def clean(df):
    # clean data
    for name in ['Longitude','Latitude']:
        for stop in ['NaN',None,0,'False']:
            df = df[df[name] != stop]
        df[name] = pd.to_numeric(df[name], downcast="float")
    return df

In [ ]:
import numpy as np
# clean data
VDS_df = clean(VDS_df)
CCS_df = clean(CCS_df)

# set vds locations
from scipy import spatial
locations = []
for lat, long in zip(VDS_df['Latitude'],VDS_df['Longitude']):
    locations = locations + [(lat, long)]   
locations =  np.array(locations)
tree = spatial.KDTree(locations)

In [ ]:
# calc 10 nearest vds stations to a ccs station 
points, distance = [], []
for lat, long in zip(CCS_df['Latitude'],CCS_df['Longitude']):
    tree.query([(lat, long)])
    results = tree.query([(lat, long)], k=10)
    distance.append(results[0][0])
    points.append(results[1][0]) 
CCS_df['Geo_dist'] = distance
CCS_df['k_near'] = points

# sub-select nearest stations to the ccs
vds = []
for k10 in CCS_df['k_near']:
    vds = vds + list(k10)
vds_k10_idx = list(set(vds))
sub_vds = VDS_df[VDS_df.index.isin(vds_k10_idx)]

sub_vds['full_name'] = sub_vds['ID2'].astype(str)+' '+sub_vds['Roadway_Name']+' '+sub_vds['Direction']

In [ ]:
# --- POINT PLOT sub_VDS --- #
sub_df = CreatePointMap(sub_vds, 'full_name')
# draw points on map
GeneratePointMap(sub_df.loc[:,['full_name']], sub_df.loc[:,['Latitude']],sub_df.loc[:,['Longitude']], 'green', radius=6)